In [1]:
%%writefile /mnt/workspace/install.sh
# 切换到 conda 的环境文件夹
cd  /opt/conda/envs 
mkdir ipex
# 下载 ipex-llm 官方环境
wget https://s3.idzcn.com/ipex-llm/ipex-llm-2.1.0b20240410.tar.gz 
# 解压文件夹以便恢复原先环境
tar -zxvf ipex-llm-2.1.0b20240410.tar.gz -C ipex/ && rm ipex-llm-2.1.0b20240410.tar.gz
# 安装 ipykernel 并将其注册到 notebook 可使用内核中
/opt/conda/envs/ipex/bin/python3 -m pip install ipykernel && /opt/conda/envs/ipex/bin/python3 -m ipykernel install --name=ipex

Writing /mnt/workspace/install.sh


bash install.sh

conda activate ipex

In [1]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
# 第一个参数表示下载模型的型号，第二个参数是下载后存放的缓存地址，第三个表示版本号，默认 master
model_dir = snapshot_download('Qwen/Qwen2-1.5B-Instruct', cache_dir='qwen2chat_src', revision='master')

2024-08-02 16:43:50,721 - modelscope - INFO - PyTorch version 2.2.2 Found.
2024-08-02 16:43:50,722 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-08-02 16:43:50,722 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-08-02 16:43:50,774 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 8dabb305acc7e73440bd9a5a5a66fb34 and a total number of 972 components indexed
/opt/conda/envs/ipex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 660/660 [00:00<00:00, 102kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 8.17kB/s]
Downloading: 100%|██████████| 242/242 [00:00<00:00, 162kB/s]
Downloading: 100%|██████████

pip install ipex_llm PyMuPDF llama-index-vector-stores-chroma llama-index-readers-file llama-index-embeddings-huggingface llama-index streamlit ipex_llm

In [2]:
from ipex_llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer 
import os
if __name__ == '__main__':
    model_path = os.path.join(os.getcwd(),"qwen2chat_src/Qwen/Qwen2-1___5B-Instruct")
    model = AutoModelForCausalLM.from_pretrained(model_path, load_in_low_bit='sym_int4', trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model.save_low_bit('qwen2chat_int4')
    tokenizer.save_pretrained('qwen2chat_int4')

2024-08-02 16:44:51,650 - INFO - Converting the current model to sym_int4 format......
2024-08-02 16:44:52,065 - INFO - PyTorch version 2.2.2 available.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model_dir = snapshot_download('AI-ModelScope/bge-small-zh-v1.5', cache_dir='qwen2chat_src', revision='master')

Downloading: 100%|██████████| 124/124 [00:00<00:00, 74.8kB/s]
Downloading: 100%|██████████| 47.0/47.0 [00:00<00:00, 31.5kB/s]
Downloading: 100%|██████████| 91.4M/91.4M [00:00<00:00, 124MB/s] 
Downloading: 100%|██████████| 349/349 [00:00<00:00, 227kB/s]
Downloading: 100%|██████████| 91.4M/91.4M [00:00<00:00, 151MB/s]
Downloading: 100%|██████████| 27.5k/27.5k [00:00<00:00, 5.13MB/s]
Downloading: 100%|██████████| 52.0/52.0 [00:00<00:00, 34.5kB/s]
Downloading: 100%|██████████| 125/125 [00:00<00:00, 81.6kB/s]
Downloading: 100%|██████████| 429k/429k [00:00<00:00, 31.7MB/s]
Downloading: 100%|██████████| 367/367 [00:00<00:00, 251kB/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 7.78MB/s]


In [16]:
%%writefile /mnt/workspace/run_rag.py
# 设置OpenMP线程数为8
import os
import time
os.environ["OMP_NUM_THREADS"] = "8"

import torch
from typing import Any, List, Optional, Iterator

from transformers import TextStreamer, AutoTokenizer

# 从llama_index库导入HuggingFaceEmbedding类，用于将文本转换为向量表示
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# 从llama_index库导入ChromaVectorStore类，用于高效存储和检索向量数据
from llama_index.vector_stores.chroma import ChromaVectorStore
# 从llama_index库导入PyMuPDFReader类，用于读取和解析PDF文件内容
from llama_index.readers.file import PyMuPDFReader
# 从llama_index库导入NodeWithScore和TextNode类
# NodeWithScore: 表示带有相关性分数的节点，用于排序检索结果
# TextNode: 表示文本块，是索引和检索的基本单位。节点存储文本内容及其元数据，便于构建知识图谱和语义搜索
from llama_index.core.schema import NodeWithScore, TextNode
# 从llama_index库导入RetrieverQueryEngine类，用于协调检索器和响应生成，执行端到端的问答过程()
from llama_index.core.query_engine import RetrieverQueryEngine
# 从llama_index库导入QueryBundle类，用于封装查询相关的信息，如查询文本、过滤器等
from llama_index.core import QueryBundle
# 从llama_index库导入BaseRetriever类，这是所有检索器的基类，定义了检索接口
from llama_index.core.retrievers import BaseRetriever
# 从llama_index库导入SentenceSplitter类，用于将长文本分割成句子或语义完整的文本块，便于索引和检索
from llama_index.core.node_parser import SentenceSplitter
# 从llama_index库导入VectorStoreQuery类，用于构造向量存储的查询，支持语义相似度搜索
from llama_index.core.vector_stores import VectorStoreQuery
# 向量数据库
import chromadb
from ipex_llm.llamaindex.llms import IpexLLM

class Config:
    """配置类,存储所有需要的参数"""
    model_path = "qwen2chat_int4"
    tokenizer_path = "qwen2chat_int4"
    question = "根据道德经回答问题"
    data_path = "./data/Full text of Tao Te Ching.pdf"
    persist_dir = "./chroma_db"
    embedding_model_path = "qwen2chat_src/AI-ModelScope/bge-small-zh-v1___5"
    max_new_tokens = 64
    
    def set_question(cls, question):
        cls.question = question
        
def load_vector_database(persist_dir: str) -> ChromaVectorStore:
    """
    加载或创建向量数据库
    
    Args:
        persist_dir (str): 持久化目录路径
    
    Returns:
        ChromaVectorStore: 向量存储对象
    """
    # 检查持久化目录是否存在
    if os.path.exists(persist_dir):
        print(f"正在加载现有的向量数据库: {persist_dir}")
        chroma_client = chromadb.PersistentClient(path=persist_dir)
        chroma_collection = chroma_client.get_collection("llama2_paper")
    else:
        print(f"创建新的向量数据库: {persist_dir}")
        chroma_client = chromadb.PersistentClient(path=persist_dir)
        chroma_collection = chroma_client.create_collection("llama2_paper")
    print(f"Vector store loaded with {chroma_collection.count()} documents")
    return ChromaVectorStore(chroma_collection=chroma_collection)

def load_data(data_path: str) -> List[TextNode]:
    """
    加载并处理PDF数据
    
    Args:
        data_path (str): PDF文件路径
    
    Returns:
        List[TextNode]: 处理后的文本节点列表
    """
    loader = PyMuPDFReader()
    documents = loader.load(file_path=data_path)

    text_parser = SentenceSplitter(chunk_size=384)
    text_chunks = []
    doc_idxs = []
    for doc_idx, doc in enumerate(documents):
        cur_text_chunks = text_parser.split_text(doc.text)
        text_chunks.extend(cur_text_chunks)
        doc_idxs.extend([doc_idx] * len(cur_text_chunks))

    nodes = []
    for idx, text_chunk in enumerate(text_chunks):
        node = TextNode(text=text_chunk)
        src_doc = documents[doc_idxs[idx]]
        node.metadata = src_doc.metadata
        nodes.append(node)
    return nodes

class VectorDBRetriever(BaseRetriever):
    """向量数据库检索器"""

    def __init__(
        self,
        vector_store: ChromaVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """
        检索相关文档
        
        Args:
            query_bundle (QueryBundle): 查询包
        
        Returns:
            List[NodeWithScore]: 检索到的文档节点及其相关性得分
        """
        query_embedding = self._embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = self._vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))
        print(f"Retrieved {len(nodes_with_scores)} nodes with scores")
        return nodes_with_scores

def completion_to_prompt(completion: str) -> str:
    """
    将完成转换为提示格式
    
    Args:
        completion (str): 完成的文本
    
    Returns:
        str: 格式化后的提示
    """
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

def messages_to_prompt(messages: List[dict]) -> str:
    """
    将消息列表转换为提示格式
    
    Args:
        messages (List[dict]): 消息列表
    
    Returns:
        str: 格式化后的提示
    """
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    prompt = prompt + "<|assistant|>\n"

    return prompt

def setup_llm(config: Config) -> IpexLLM:
    """
    设置语言模型
    
    Args:
        config (Config): 配置对象
    
    Returns:
        IpexLLM: 配置好的语言模型
    """
    return IpexLLM.from_model_id_low_bit(
        model_name=config.model_path,
        tokenizer_name=config.tokenizer_path,
        context_window=384,
        max_new_tokens=config.max_new_tokens,
        generate_kwargs={"temperature": 0.7, "do_sample": False},
        model_kwargs={},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="cpu",
    )

def predict()-> Iterator[str]:
    config = Config()
    
    # 设置嵌入模型
    embed_model = HuggingFaceEmbedding(model_name=config.embedding_model_path)
    
    # 设置语言模型
    llm = setup_llm(config)
    
    # 加载向量数据库
    vector_store = load_vector_database(persist_dir=config.persist_dir)
    
    # 加载和处理数据
    nodes = load_data(data_path=config.data_path)
    for node in nodes:
        node_embedding = embed_model.get_text_embedding(
            node.get_content(metadata_mode="all")
        )
        node.embedding = node_embedding
    
    # 将 node 添加到向量存储
    vector_store.add(nodes)
    
    # 设置查询
    query_str = config.question
    query_embedding = embed_model.get_query_embedding(query_str)
    
    # 执行向量存储检索
    query_mode = "default"
    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)

    # 处理查询结果
    nodes_with_scores = []
    for index, node in enumerate(query_result.nodes):
        score: Optional[float] = None
        if query_result.similarities is not None:
            score = query_result.similarities[index]
        nodes_with_scores.append(NodeWithScore(node=node, score=score))
    
    # 设置检索器
    retriever = VectorDBRetriever(
        vector_store, embed_model, query_mode="default", similarity_top_k=1
    )    
    
    # 加载对应的tokenizer ~~
    tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_path, trust_remote_code=True)    
    
    # 创建文本流式输出器 ~~
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
    # 创建查询引擎
    query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm, streaming=True)

    # 执行查询
#    start_time = time.time()
    streaming_response = query_engine.query(query_str)
    
    for token in streaming_response.response_gen:
        yield token
     
    
def main():
    """主函数"""
    config = Config()
    
    # 设置嵌入模型
    embed_model = HuggingFaceEmbedding(model_name=config.embedding_model_path)
    
    # 设置语言模型
    llm = setup_llm(config)
    
    # 加载向量数据库
    vector_store = load_vector_database(persist_dir=config.persist_dir)
    
    # 加载和处理数据
    nodes = load_data(data_path=config.data_path)
    for node in nodes:
        node_embedding = embed_model.get_text_embedding(
            node.get_content(metadata_mode="all")
        )
        node.embedding = node_embedding
    
    # 将 node 添加到向量存储
    vector_store.add(nodes)
    
    # 设置查询
    query_str = config.question
    query_embedding = embed_model.get_query_embedding(query_str)
    
    # 执行向量存储检索
    print("开始执行向量存储检索")
    query_mode = "default"
    vector_store_query = VectorStoreQuery(
        query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
    )
    query_result = vector_store.query(vector_store_query)

    # 处理查询结果
    print("开始处理检索结果")
    nodes_with_scores = []
    for index, node in enumerate(query_result.nodes):
        score: Optional[float] = None
        if query_result.similarities is not None:
            score = query_result.similarities[index]
        nodes_with_scores.append(NodeWithScore(node=node, score=score))
    
    # 设置检索器
    retriever = VectorDBRetriever(
        vector_store, embed_model, query_mode="default", similarity_top_k=1
    )
    
    print(f"Query engine created with retriever: {type(retriever).__name__}")
    print(f"Query string length: {len(query_str)}")
    print(f"Query string: {query_str}")
    
    # 创建查询引擎
    print("准备与llm对话")
    query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

    # 执行查询
    print("开始RAG最后生成")
    start_time = time.time()
    response = query_engine.query(query_str)

    # 打印结果
    print("------------RESPONSE GENERATION---------------------")
    print(str(response))
    print(f"inference time: {time.time()-start_time}")

if __name__ == "__main__":
    main()

Overwriting /mnt/workspace/run_rag.py


In [18]:
%%writefile /mnt/workspace/run_streamlit_stream.py


# 导入操作系统模块，用于设置环境变量
import os
# 设置环境变量 OMP_NUM_THREADS 为 8，用于控制 OpenMP 线程数
os.environ["OMP_NUM_THREADS"] = "8"

# 导入时间模块
import time
# 导入 Streamlit 模块，用于创建 Web 应用
import streamlit as st
# 从 transformers 库中导入 AutoTokenizer 类
from transformers import AutoTokenizer
# 从 transformers 库中导入 TextStreamer 类
from transformers import TextStreamer, TextIteratorStreamer
# 从 ipex_llm.transformers 库中导入 AutoModelForCausalLM 类
from ipex_llm.transformers import AutoModelForCausalLM
# 导入 PyTorch 库
import torch
from threading import Thread

import run_rag
def generate_response(messages, message_placeholder):
    run_rag.Config.set_question(run_rag.Config, messages[-1]["content"])
    # run_rag.Config.set_question(run_rag.Config,"天地不仁，以万物为刍狗。")
    return run_rag.predict()

# Streamlit 应用部分
# 设置应用标题
st.title("对话道德经")

# 初始化聊天历史，如果不存在则创建一个空列表
if "messages" not in st.session_state:
    st.session_state.messages = []

# 显示聊天历史
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 用户输入部分
if prompt := st.chat_input("你想说点什么?"):
    # 将用户消息添加到聊天历史
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    response  = str()
    # 创建空的占位符用于显示生成的响应
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        
        # 调用模型生成响应
        streamer = generate_response(st.session_state.messages, message_placeholder)
        for text in streamer:
            response += text
            message_placeholder.markdown(response + "▌")
    
        message_placeholder.markdown(response)
    
    # 将助手的响应添加到聊天历史
    st.session_state.messages.append({"role": "assistant", "content": response})


Overwriting /mnt/workspace/run_streamlit_stream.py


streamlit run run_streamlit_stream.py